# Dataset Info.

### train.csv [파일]

- ID : 실제 판매되고 있는 고유 ID
- 제품 : 제품 코드
- 대분류 : 제품의 대분류 코드
- 중분류 : 제품의 중분류 코드
- 소분류 : 제품의 소분류 코드
- 브랜드 : 제품의 브랜드 코드
- 2022-01-01 ~ 2023-04-04 : 실제 일별 판매량
- 단, 제품이 동일하여도 판매되고 있는 고유 ID 별로 기재한 분류 정보가 상이할 수 있음
- 즉 고유 ID가 다르다면, 제품이 같더라도 다른 판매 채널


### sample_submission.csv [파일] - 제출 양식
- ID : 실제 판매되고 있는 고유 ID
- 2023-04-05 ~ 2023-04-25 : 예측한 일별 판매량


### sales.csv [파일] - 메타(Meta) 정보
- ID : 실제 판매되고 있는 고유 ID
- 제품 : 제품 코드
- 대분류 : 제품의 대분류 코드
- 중분류 : 제품의 중분류 코드
- 소분류 : 제품의 소분류 코드
- 브랜드 : 제품의 브랜드 코드
- 2022-01-01 ~ 2023-04-04 : 실제 일별 총 판매금액
- 단, 제품이 동일하여도 판매되고 있는 고유 ID 별로 기재한 분류 정보가 상이할 수 있음
- 즉 고유 ID가 다르다면, 제품이 같더라도 다른 판매 채널


### brand_keyword_cnt.csv [파일] - 메타(Meta) 정보
- 브랜드 : 브랜드 코드
- 2022-01-01 ~ 2023-04-04 : 브랜드의 연관키워드 언급량을 정규화한 일별 데이터


### product_info.csv [파일] - 메타(Meta) 정보
- 제품 : 제품 코드
- 제품특성 : 제품 특성 데이터(Text)
- train.csv에 존재하는 모든 제품 코드가 포함되어 있지 않음. 또는 product_info.csv에 존재하는 제품 코드가 train.csv에 존재하지 않을 수 있음


In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
filepath = '/Users/leeshinhaeng/Desktop/open'
filelist = os.listdir(filepath)

In [3]:
filelist

['product_info.csv',
 'brand_keyword_cnt.csv',
 'train.csv',
 'sample_submission.csv',
 'sales.csv']

In [4]:
# product_info
info = pd.read_csv(filepath+'/'+filelist[0]) # 제품정보
# brand_keyword_cnt
kw = pd.read_csv(filepath+'/'+filelist[1]) # 브랜드별 언급량
# train_data
train = pd.read_csv(filepath+'/'+filelist[2]) # 훈련데이터
# sample_submission
submissions = pd.read_csv(filepath+'/'+filelist[3]) # 제출
# sales
sales = pd.read_csv(filepath+'/'+filelist[4]) # 판매량

In [5]:
# 제품 특성 데이터
info.head(3)

,제품,제품특성
0,B002-03509-00001,제품유형:일반식품 콜라겐 펩타이드:1000mg 종류:어류 분자량:300Da 섭취대상...
1,B002-02376-00001,700mg x 28정
2,B002-03735-00001,제품타입:정 섭취방법:물과 함께 섭취대상:성인남녀 섭취횟수:하루 두 번 1일 총 섭...


모델링 시 info데이터는 활용하지 못함. 왜냐하면, 예측해야 하는게 15890개인데, 12778개에 대한 정보만 제공되어 있음

In [6]:
kw.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.84131,0.91383,1.450530,2.422390,1.871190,1.581080,1.232950,1.174930,1.145920,...,0.319110,0.391640,0.377130,0.49318,0.072520,0.29010,0.31911,0.232080,0.333620,0.44966
1,B002-00002,12.64868,20.27850,15.332170,12.750210,13.562510,13.707570,11.937910,15.564250,14.084710,...,10.269790,11.966920,10.646930,10.41485,10.487380,9.48651,9.28343,10.429350,11.154620,11.38671
2,B002-00003,0.33362,0.43516,0.362630,0.174060,0.217580,0.464170,0.420650,0.290100,0.377130,...,0.536690,0.696250,0.449660,0.39164,1.029880,0.49318,0.91383,0.797790,1.015370,0.88482
3,B002-00005,1.07339,1.71163,2.016240,1.914700,1.987230,2.146790,1.682620,1.378000,1.421520,...,2.219320,2.509420,2.872060,2.37888,2.030750,1.53756,1.34899,1.261960,2.320850,2.30635
4,B002-00006,0.00000,0.00000,0.188558,0.246574,0.246574,0.246574,0.377139,0.087012,0.261084,...,0.072526,0.290103,0.087012,0.00000,0.130542,0.00000,0.00000,0.072526,0.217577,0.00000


In [28]:
# 일별 판매량 데이터 - train
train_split = train.drop(['제품','대분류','중분류','소분류','브랜드'], axis=1)
product_split = train[['ID','대분류','중분류','소분류','브랜드']]

In [29]:
# 기타 칼럼들을 ID와 날짜 칼럼에 해당되게끔 행으로 옮기는 작업 수행
# 이 경우 데이터가 700만건이 넘게 발생
melted_train = train_split.melt(id_vars=['ID'], var_name='Date', value_name='Value')

In [30]:
melted_train.head()

,ID,Date,Value
0,0,2022-01-01,0
1,1,2022-01-01,0
2,2,2022-01-01,0
3,3,2022-01-01,0
4,4,2022-01-01,0


In [31]:
melted_train['Date'] = pd.to_datetime(melted_train['Date'])
melted_train.sort_values(by=['ID','Date'], inplace=True)
melted_train.reset_index(drop=True, inplace=True)

In [32]:
# 데이터 합치기
melted_train = pd.merge(melted_train, product_split, on='ID')

In [33]:
melted_train.head()

,ID,Date,Value,대분류,중분류,소분류,브랜드
0,0,2022-01-01,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001
1,0,2022-01-02,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001
2,0,2022-01-03,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001
3,0,2022-01-04,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001
4,0,2022-01-05,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001


In [13]:
# 판매 금액
sales.head()

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,22400,67200,44800,0,0,44800,0
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
sales_split = train.drop(['제품','대분류','중분류','소분류','브랜드'], axis=1)
sales_split.head()

,ID,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
melted_sales = sales_split.melt(id_vars=['ID'], var_name='Date', value_name='Value')
melted_sales['Date'] = pd.to_datetime(melted_train['Date'])
melted_sales.sort_values(by=['ID','Date'], inplace=True)
melted_sales.reset_index(drop=True, inplace=True)

In [16]:
melted_sales

,ID,Date,Value
0,0,2022-01-01,0
1,0,2022-01-02,2
2,0,2022-01-03,0
3,0,2022-01-04,0
4,0,2022-01-05,0
...,...,...,...
7293505,15889,2023-03-31,0
7293506,15889,2023-04-01,0
7293507,15889,2023-04-02,0
7293508,15889,2023-04-03,0


In [17]:
melted_train = pd.merge(melted_train, melted_sales, on=['ID','Date'])

In [18]:
melted_train

,ID,Date,Value_x,대분류,중분류,소분류,브랜드,Value_y
0,0,2022-01-01,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0
1,0,2022-01-02,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2
2,0,2022-01-03,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0
3,0,2022-01-04,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0
4,0,2022-01-05,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0
...,...,...,...,...,...,...,...,...
7293505,15889,2023-03-31,0,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,0
7293506,15889,2023-04-01,0,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,0
7293507,15889,2023-04-02,0,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,0
7293508,15889,2023-04-03,0,B002-C001-0002,B002-C002-0004,B002-C003-0020,B002-03799,0


In [19]:
melted_train.rename(columns = {'Value_x' : 'count_vlaue', 'Value_y' : 'money_value'}, inplace=True)

In [20]:
melted_train.head()

,ID,Date,count_vlaue,대분류,중분류,소분류,브랜드,money_value
0,0,2022-01-01,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0
1,0,2022-01-02,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2
2,0,2022-01-03,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0
3,0,2022-01-04,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0
4,0,2022-01-05,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0


In [21]:
melted_train['브랜드'].nunique()

3170

In [22]:
kw

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.84131,0.91383,1.450530,2.422390,1.871190,1.581080,1.232950,1.174930,1.145920,...,0.319110,0.391640,0.377130,0.49318,0.072520,0.29010,0.31911,0.232080,0.333620,0.44966
1,B002-00002,12.64868,20.27850,15.332170,12.750210,13.562510,13.707570,11.937910,15.564250,14.084710,...,10.269790,11.966920,10.646930,10.41485,10.487380,9.48651,9.28343,10.429350,11.154620,11.38671
2,B002-00003,0.33362,0.43516,0.362630,0.174060,0.217580,0.464170,0.420650,0.290100,0.377130,...,0.536690,0.696250,0.449660,0.39164,1.029880,0.49318,0.91383,0.797790,1.015370,0.88482
3,B002-00005,1.07339,1.71163,2.016240,1.914700,1.987230,2.146790,1.682620,1.378000,1.421520,...,2.219320,2.509420,2.872060,2.37888,2.030750,1.53756,1.34899,1.261960,2.320850,2.30635
4,B002-00006,0.00000,0.00000,0.188558,0.246574,0.246574,0.246574,0.377139,0.087012,0.261084,...,0.072526,0.290103,0.087012,0.00000,0.130542,0.00000,0.00000,0.072526,0.217577,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,B002-03794,2.32085,2.98810,3.611830,4.061500,3.669850,3.771390,3.031620,2.988100,3.133150,...,2.422390,2.422390,2.756010,2.32085,2.088770,1.98723,1.07339,1.929210,2.509420,1.78416
3166,B002-03795,0.14505,0.00000,0.087030,0.072520,0.087030,0.101530,0.072520,0.130540,0.116040,...,0.000000,0.072520,0.000000,0.10153,0.101530,0.00000,0.00000,0.000000,0.000000,0.00000
3167,B002-03796,0.00000,0.00000,0.000000,0.000000,0.000000,0.101530,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.072520,0.07252
3168,B002-03798,0.14505,0.00000,0.116040,0.072520,0.116040,0.275600,0.217580,0.116040,0.101530,...,0.101530,0.087030,0.145050,0.17406,0.188560,0.11604,0.11604,0.087030,0.174060,0.10153


In [23]:
melted_kw = kw.melt(id_vars=['브랜드'], var_name='Date', value_name='Value')
melted_kw['Date'] = pd.to_datetime(melted_kw['Date'])
melted_kw.sort_values(by=['브랜드','Date'], inplace=True)
melted_kw.reset_index(drop=True, inplace=True)

In [24]:
melted_kw

,브랜드,Date,Value
0,B002-00001,2022-01-01,0.84131
1,B002-00001,2022-01-02,0.91383
2,B002-00001,2022-01-03,1.45053
3,B002-00001,2022-01-04,2.42239
4,B002-00001,2022-01-05,1.87119
...,...,...,...
1455025,B002-03799,2023-03-31,5.51203
1455026,B002-03799,2023-04-01,3.52480
1455027,B002-03799,2023-04-02,4.03249
1455028,B002-03799,2023-04-03,5.88917


In [25]:
melted_train = pd.merge(melted_train, melted_kw, on=['브랜드','Date'])

In [26]:
submissions

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15886,15886,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15887,15887,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15888,15888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 주의사항
- 검증시에는 money_value(판매금액) 정보와, Value(브랜드 연관키워드 언급량)가 없는 상태임

In [27]:
melted_train.columns

Index(['ID', 'Date', 'count_vlaue', '대분류', '중분류', '소분류', '브랜드', 'money_value',
       'Value'],
      dtype='object')

In [37]:
melted_train

,ID,Date,count_vlaue,대분류,중분류,소분류,브랜드,money_value,Value
0,0,2022-01-01,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0.84131
1,0,2022-01-02,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2,0.91383
2,0,2022-01-03,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,1.45053
3,0,2022-01-04,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,2.42239
4,0,2022-01-05,0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,1.87119
...,...,...,...,...,...,...,...,...,...
7293505,15885,2023-04-04,0,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,3,5.07687
7293506,15886,2023-04-04,3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,0,5.07687
7293507,15887,2023-04-04,0,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,13,5.07687
7293508,15888,2023-04-04,2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,5,5.07687
